In [7]:
import csv
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
sns.set_style('ticks')
pd.set_option('display.max_columns', None)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Introduction and problem statement
<hr>

This project analyzes a comprehensive housing dataset from Ames, Iowa, USA ([source](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)). Homeowners often overspend on remodeling projects, which can decrease the return on their investment when selling their home. Conversely, potential buyers want to maximize the value of their home purchase within their budget. This project aims to address these concerns by investigating the following questions:

- What features contribute the most and the least to a house's value?
- Given a set of features, what is the expected selling price of a house?
- Given a budget, what houses are affordable?

To answer these questions, various linear regression models were fitted to the housing data to identify the most and least impactful features. The performance of the models will be evaluated using the R2 metric, which measures how well the model can explain the variance in the dataset.

## Table of contents

1. [Introduction and problem statement](#Introduction-and-problem-statement)
2.
3.
4.
5.

In [2]:
# import train
train = pd.read_csv ("data/train.csv")

# make columns lowercase and remove spaces
train.columns = train.columns.str.lower()
train.columns = train.columns.str.replace(' ', '')

print(train.columns)

Index(['id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'alley', 'lotshape', 'landcontour', 'utilities', 'lotconfig',
       'landslope', 'neighborhood', 'condition1', 'condition2', 'bldgtype',
       'housestyle', 'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd',
       'roofstyle', 'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype',
       'masvnrarea', 'exterqual', 'extercond', 'foundation', 'bsmtqual',
       'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1',
       'bsmtfintype2', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating',
       'heatingqc', 'centralair', 'electrical', '1stflrsf', '2ndflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual',
       'totrmsabvgrd', 'functional', 'fireplaces', 'fireplacequ', 'garagetype',
       'garageyrblt', 'garagefinish', 'garagecars', 'garagearea', 'garagequal',
       'garagecond', 'paveddrive

In [3]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18  overallc

In [4]:
train.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
def data_summaries(df):
    """
    Prints summaries for each column in a given DataFrame.

    For string columns, the function prints the count of unique values.

    For numerical columns, the function prints the percentage of zeros in the column.
    If the percentage of zeros is greater than or equal to 50%, the function prints
    the percentage of zeros and the number of zero values in the column. Otherwise,
    the function prints only the percentage of zeros.
     
    df (pandas.DataFrame): A pandas DataFrame to summarize.
    """
    for col in df.columns:
        print(f"------ {col} ------")
        if df[col].dtype == np.object:
            print(df[col].value_counts())
        elif len(df[df[col] == 0]) >= 731:
            print(f"lots of zeros: {round((len(df[df[col] == 0]) / 1460) * 100)}% \nno. zeros: {len(df[df[col] == 0])}")
        else:
            print(f"zeros: {round((len(df[df[col] == 0]) / 1460) * 100)}%")
        print("\n")
    
data_summaries(train)

------ id ------
zeros: 0%


------ mssubclass ------
zeros: 0%


------ mszoning ------
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: mszoning, dtype: int64


------ lotfrontage ------
zeros: 0%


------ lotarea ------
zeros: 0%


------ street ------
Pave    1454
Grvl       6
Name: street, dtype: int64


------ alley ------
Grvl    50
Pave    41
Name: alley, dtype: int64


------ lotshape ------
Reg    925
IR1    484
IR2     41
IR3     10
Name: lotshape, dtype: int64


------ landcontour ------
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: landcontour, dtype: int64


------ utilities ------
AllPub    1459
NoSeWa       1
Name: utilities, dtype: int64


------ lotconfig ------
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: lotconfig, dtype: int64


------ landslope ------
Gtl    1382
Mod      65
Sev      13
Name: landslope, dtype: int64


------ neighborhood ------
NAmes      225
CollgCr    150
OldTo

## Summary statistics
<hr>

The dataset contains information on 1460 houses constructed between 1872 and 2010 (`yearbuilt`). The `grlivarea` variable, representing the living area, had an average size of 1515 square feet (sf) with a standard deviation of 500 sf. The smallest house in the dataset had a living area of 334 sf, while the largest was 5642 sf.

For houses with basements, the `totalbsmtsf` variable had an average area of 1057 sf with a standard deviation of 449 sf. The smallest basement area was 0 sf, indicating houses without basements, while the largest was 6110 sf.

The `garagearea` variable, representing the garage area, had an average size of 473 sf with a standard deviation of 216 sf. The smallest garage was 0 sf, indicating houses without a garage, while the largest was 1418 sf, which was even bigger than the smallest house.

On average, the houses had 6.52 rooms (`totrmsabvgrd`). The number of rooms in the smallest house was 2, while the largest house had 14.

Finally, the average sale price (`saleprice`) of the houses was \\$180,921 (presumably in US dollars) with a standard deviation of \\$79,442. The cheapest house sold during this period was \\$34,900, while the most expensive was \\$755,000.

In [6]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
mssubclass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
lotfrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
lotarea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
overallqual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
overallcond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
yearbuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
yearremodadd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
masvnrarea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
bsmtfinsf1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


## Null values
<hr> 

The dataset comprises 19 columns, each with a varying number of null values ranging from 1 to 1453.

Upon closer inspection of the categorical columns and their unique values compared to the data dictionary, it appears that the null values represent houses that lack specific features such as a garage, pool, or fireplace. Similarly, the missing values in certain numerical columns indicate houses without a masonry veneer, resulting in null values for the masvnrarea variable. To address this, the nulls in the categorical columns will be filled with "None", and those in numerical columns will be filled with 0.

Additionally, some columns contain only 1 to 8 null values, which are likely recording errors. As a result, the rows with these null values will be dropped from the dataset.

In [9]:
# print count of nulls for columns containing nulls
nulldf = pd.DataFrame(train.isnull().sum(), columns = ['nullcount'])
nulldf = nulldf[nulldf['nullcount'] != 0]
print(nulldf.shape)
nulldf

(19, 1)


,nullcount
lotfrontage,259
alley,1369
masvnrtype,8
masvnrarea,8
bsmtqual,37
bsmtcond,37
bsmtexposure,38
bsmtfintype1,37
bsmtfintype2,38
electrical,1
